In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(maftools)

In [4]:
%%R
my_data = data.frame(pos = sample.int(912, 15, replace = TRUE), count = sample.int(30, 15, replace = TRUE))
head(my_data)

  pos count
1 262    24
2 119    24
3 820    27
4 165    26
5 710     6
6 810    17


In [ ]:
%%R
pdf("lollipop_plot.pdf")
lollipopPlot(data = my_data, gene = "DNMT3A")
dev.off()

     HGNC refseq.ID protein.ID aa.length
1: DNMT3A NM_022552  NP_072046       912
2: DNMT3A NM_153759  NP_715640       723
3: DNMT3A NM_175629  NP_783328       912
quartz_off_screen 
                2 


3 transcripts available. Use arguments refSeqID or proteinID to manually specify tx name.
Using longer transcript NM_022552 for now.
